In [6]:
pip install konlpy

Note: you may need to restart the kernel to use updated packages.


In [7]:
import konlpy

In [10]:
import pandas as pd

train = pd.read_csv(r'C:\Users\suhye\Desktop\Machine Learning\Naver\ratings_train.txt',sep='\t')
train.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [11]:
train['label'].value_counts()

0    75173
1    74827
Name: label, dtype: int64

In [13]:
import re
train = train.fillna(' ')

#정규 표현식을 이용해 숫자를 공백으로 변경(정규표현식으로 \d는 숫자 의미)
train['document'] = train['document'].apply(lambda x : re.sub(r"\d+"," ",x))

#테스트 데이터 셋을 로딩하고 동일하게 Null과 숫자를 공백으로 변환
test = pd.read_csv(r"C:\Users\suhye\Desktop\Machine Learning\Naver\ratings_test.txt",sep='\t')
test = test.fillna(' ')
test['document'] = test['document'].apply(lambda x : re.sub(r"\d+"," ",x))

#id컬럼 삭제 수행
train.drop('id',axis=1,inplace=True)
test.drop('id',axis=1, inplace=True)

In [16]:
from konlpy.tag import Twitter

twitter = Twitter()
def tw_tokenizer(text):
    #입력 인자로 들어온 텍스트를 형태소 단어로 토큰화하여 리스트 형태로 반환
    tokens_ko = twitter.morphs(text)
    return tokens_ko

C:\Users\suhye\anaconda3\lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

#Twitter 객체의 morphs()객체를 이용한 tokenizer 사용
tfidf_vect = TfidfVectorizer(tokenizer = tw_tokenizer,ngram_range=(1,2),min_df=3,max_df=0.9)
tfidf_matrix_train = tfidf_vect.fit_transform(train['document'])

In [19]:
lr = LogisticRegression()
params = {'C':[1,3.5,4.5,5.5,10]}
grid = GridSearchCV(lr,param_grid=params,cv=3,scoring='accuracy',verbose=1)
grid.fit(tfidf_matrix_train,train['label'])
print("best params : {}".format(grid.best_params_))
print("best score : {:.3f}".format(grid.best_score_))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


C:\Users\suhye\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\suhye\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

best params : {'C': 4.5}
best score : 0.859


C:\Users\suhye\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [21]:
from sklearn.metrics import accuracy_score

#TfidfVectorizer를 이용해 테스트 데이터를 TF-IDF 값으로 피처 변환
tfidf_matrix_test = tfidf_vect.transform(test['document'])

#classifier는 최적 classifier를 이용
best_estimator = grid.best_estimator_
preds = best_estimator.predict(tfidf_matrix_test)

print("Logistic Regression 정확도 : ",accuracy_score(test['label'],preds))

Logistic Regression 정확도 :  0.86178
